In [179]:
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report as cr, confusion_matrix as cm
from scipy.io import loadmat
from scipy.spatial.distance import pdist

In [180]:
path= "/Users/david/Desktop/AppliedDS/proj3/Spring2020-Project3-group2/data/train_set/"
point_path = path+ "points/"
labels = pd.read_csv(path+"label.csv")

In [181]:
def point_load(point_path):
    point_list = []
    for i in sorted(os.listdir(point_path)):
        point_list.append(loadmat(os.path.join(point_path ,i)))
    
    fp_list = []
    for i in range(len(point_list)):
        if 'faceCoordinatesUnwarped' in point_list[i].keys():
            fp_list.append(point_list[i]['faceCoordinatesUnwarped'])
        else:
            fp_list.append(point_list[i]['faceCoordinates2'])
    feature = np.array([pdist(x) for x in fp_list])
    return feature

In [251]:
from keras.utils import to_categorical, normalize

X = point_load(point_path)
y = labels["emotion_idx"]

#pca = PCA(n_components=0.99, whiten=True)
#X_pca_train = pca.fit_transform(X_train)
#X_pca_test = pca.transform(X_test)


In [353]:
X.shape

(2500, 3003)

In [254]:

ss = StandardScaler()
X = ss.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y-1, test_size = .2, random_state = 123)

In [414]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.regularizers import l1, l2
from tensorflow.keras.optimizers import Adam

#build layers

model = Sequential()
model.add(Dense(800,input_dim = 3003, activation="relu",
                kernel_initializer = 'glorot_normal', 
                activity_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(.2))
model.add(Dense(300, activation="relu",kernel_initializer = 'glorot_normal'))
model.add(Dropout(.2))
model.add(Dense(100, activation="relu",kernel_initializer = 'glorot_normal'))
model.add(Dropout(.2))
model.add(Dense(22, activation="sigmoid"))



In [418]:
# run model
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=.001), metrics=['accuracy'])
model.fit(X_train, 
          y_train, 
          epochs=40, 
          batch_size = 32,
          validation_split=.2, 
          validation_data = (X_test, y_test))

Train on 2000 samples, validate on 500 samples
Epoch 1/40
2000/2000 [==============================] - 1s 605us/step - loss: 9.2941 - accuracy: 0.1280 - val_loss: 5.9303 - val_accuracy: 0.2520
Epoch 2/40
2000/2000 [==============================] - 1s 448us/step - loss: 6.6966 - accuracy: 0.2380 - val_loss: 7.0902 - val_accuracy: 0.2620
Epoch 3/40
2000/2000 [==============================] - 1s 466us/step - loss: 7.3100 - accuracy: 0.2830 - val_loss: 7.4920 - val_accuracy: 0.3400
Epoch 4/40
2000/2000 [==============================] - 1s 458us/step - loss: 7.2546 - accuracy: 0.3075 - val_loss: 9.3081 - val_accuracy: 0.3240
Epoch 5/40
2000/2000 [==============================] - 1s 483us/step - loss: 6.5174 - accuracy: 0.3485 - val_loss: 6.4745 - val_accuracy: 0.3440
Epoch 6/40
2000/2000 [==============================] - 1s 475us/step - loss: 6.4886 - accuracy: 0.3755 - val_loss: 8.3517 - val_accuracy: 0.3640
Epoch 7/40
2000/2000 [==============================] - 1s 488us/step - loss:

In [419]:
_, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 41.40


In [413]:
predictions = model.predict(X_test[:10])

In [410]:

print(np.argmax(predictions, axis = 1))

[ 4 15 21 21  2]


In [412]:
print(y_test[:10])

2386    20
1298    11
1995    17
2097    18
289      2
2380    20
2336    20
1492    13
33       0
1970    17
Name: emotion_idx, dtype: int64
